In [1]:
from stellargraph.core import StellarGraph
from stellargraph.mapper import AdjacencyPowerGenerator
from stellargraph.layer.watch_your_step import WatchYourStep, get_embeddings
from stellargraph.losses import graph_log_likelihood
from stellargraph import datasets
from stellargraph.utils import plot_history

from matplotlib import pyplot as plt
from tensorflow.keras import optimizers, Model, layers, regularizers
import tensorflow as tf
from sklearn import preprocessing, feature_extraction, model_selection
from IPython.display import display, HTML

import networkx as nx
import random
import numpy as np
import pandas as pd
import os

In [2]:
tf.random.set_seed(1234)

## Loading in the data

In [3]:
dataset = datasets.Cora()
display(HTML(dataset.description))
G, subjects = dataset.load()

## Creating the model

We create an `AdjacencyPowerGenerator` which loops through the rows of the first `num_powers` of the adjacency matrix.

In [4]:
generator = AdjacencyPowerGenerator(G, num_powers=10)

Next, we use the `WatchYourStep` class to create trainable node embeddings and expected random walks.

In [5]:
wys = WatchYourStep(
    generator,
    num_walks=80,
    embedding_dimension=128,
    attention_regularizer=regularizers.l2(0.5),
)
x_in, x_out = wys.build()

We use the graph log likelihood as our loss function.

In [6]:
model = Model(inputs=x_in, outputs=x_out)
model.compile(loss=graph_log_likelihood, optimizer=tf.keras.optimizers.Adam(1e-3))

## Training

We now create a training generator and fit our model.

In [7]:
epochs = 100

In [8]:
batch_size = 10
train_gen = generator.flow(batch_size=batch_size, threads=10)

history = model.fit(
    train_gen, epochs=epochs, verbose=1, steps_per_epoch=int(len(G.nodes()) // batch_size)
)

Train for 270 steps
Epoch 1/100
270/270 [==============================] - 2s 6ms/step - loss: 186700.2018
Epoch 2/100
270/270 [==============================] - 1s 5ms/step - loss: 178969.1792
Epoch 3/100
270/270 [==============================] - 1s 5ms/step - loss: 129431.9966
Epoch 4/100
270/270 [==============================] - 1s 5ms/step - loss: 57934.4538
Epoch 5/100
270/270 [==============================] - 1s 5ms/step - loss: 32088.0088
Epoch 6/100
270/270 [==============================] - 1s 5ms/step - loss: 28159.3023
Epoch 7/100
270/270 [==============================] - 1s 5ms/step - loss: 27210.7526
Epoch 8/100
270/270 [==============================] - 1s 5ms/step - loss: 26804.0880
Epoch 9/100
270/270 [==============================] - 1s 5ms/step - loss: 26550.2599
Epoch 10/100
270/270 [==============================] - 1s 5ms/step - loss: 26333.4867
Epoch 11/100
270/270 [==============================] - 1s 6ms/step - loss: 26084.3134
Epoch 12/100
270/270 [=======

In [9]:
plot_history(history)

## Visualizing Embeddings

Now we use TSNE to visualize the embeddings.

In [10]:
embeddings = get_embeddings(model)

In [11]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.manifold import TSNE

nodelist = list(G.nodes())

labels = subjects.loc[nodelist]
target_encoding = OneHotEncoder(sparse=False)
label_vectors = target_encoding.fit_transform(labels.values.reshape(-1, 1))

In [12]:
transform = TSNE

trans = transform(n_components=2)
emb_transformed = pd.DataFrame(trans.fit_transform(embeddings), index=nodelist)

emb_transformed["label"] = np.argmax(label_vectors, 1)

In [13]:
alpha = 0.7

fig, ax = plt.subplots(figsize=(7, 7))
ax.scatter(
    emb_transformed[0],
    emb_transformed[1],
    c=emb_transformed["label"].astype("category"),
    cmap="jet",
    alpha=alpha,
)

ax.set(aspect="equal", xlabel="$X_1$", ylabel="$X_2$")
plt.title(
    "{} visualization of Watch Your Step embeddings for cora dataset".format(
        transform.__name__
    )
)
plt.show()

# Classification

Here, we predict the class of a node by performing a weighted average of the training labels, with the weights determined by the similarity of that node's embedding with the training node embeddings.

In [14]:
# choose a random set of training nodes by permuting the labels and taking the first 300.
shuffled_idx = np.random.permutation(label_vectors.shape[0])
train_node_idx = shuffled_idx[:300]
test_node_idx = shuffled_idx[300:]

training_labels = label_vectors.copy()
training_labels[test_node_idx] = 0

In [15]:
d = embeddings.shape[1] // 2

predictions = np.dot(
    np.exp(np.dot(embeddings[:, :d], embeddings[:, d:].transpose())), training_labels
)

np.mean(
    np.argmax(predictions[test_node_idx], 1) == np.argmax(label_vectors[test_node_idx], 1)
)